Revision of concepts using an Example

1. Please note that the topic related to functions has not bee included in here.
2. There are some Financial, Numeric, Character and Date functions; pretty easy and can be googled up.
3. We might try and use some of them in our example below

Example:

1. We have been given dataset for 5 years of policy details of a car in 5 CSV formats
2. e have been given a dataset for RTO details
3. We need to combine all of those and answer certain questions

IMPORTING THE DATA

1. Let us get all the individual data sets
2. Understand their contents summary
3. Combine them with RTO data set
4. Get RTO details in final dataset

In [1]:
FILENAME file_1"/folders/myfolders/Practice_Folders_SAS/Reference Materials/Assignment/2015.csv";
FILENAME file_2"/folders/myfolders/Practice_Folders_SAS/Reference Materials/Assignment/2016.csv";
FILENAME file_3"/folders/myfolders/Practice_Folders_SAS/Reference Materials/Assignment/2017.csv";
FILENAME file_4"/folders/myfolders/Practice_Folders_SAS/Reference Materials/Assignment/2018.csv";
FILENAME file_5"/folders/myfolders/Practice_Folders_SAS/Reference Materials/Assignment/2019.csv";
FILENAME file_6"/folders/myfolders/Practice_Folders_SAS/Reference Materials/Assignment/RTO.csv";

data data_2015;
    infile file_1
        dlm=','
        firstobs = 2
        truncover
        ;
    length type $ 50;
    length manufacturer $ 50;
    length party_type $ 50;
    length car_number $ 20;
    length registration_date $ 10;
    input Policy_num mobile_num TxnID Year Type$ Base_Premium No_Claim_bonus_discount road_side_assistence_addon Tyre_addon 
          Electronic_addon thirt_party_liablity_premium manufacturer $ Party_type $ registration_date $ Car_number $
;     
data data_2016;
    infile file_2
        dlm=','
        firstobs = 2
        truncover
        ;
    length type $ 50;
    length manufacturer $ 50;
    length party_type $ 50;
    length car_number $ 20;
    length registration_date $ 10;
    input Policy_num mobile_num TxnID Year Type$ Base_Premium No_Claim_bonus_discount road_side_assistence_addon Tyre_addon 
          Electronic_addon thirt_party_liablity_premium manufacturer $ Party_type $ registration_date $ Car_number $
;   
data data_2017;
    infile file_3
        dlm=','
        firstobs = 2
        truncover
        ;
    length type $ 50;
    length manufacturer $ 50;
    length party_type $ 50;
    length car_number $ 20;
    length registration_date $ 10;
    input Policy_num mobile_num TxnID Year Type$ Base_Premium No_Claim_bonus_discount road_side_assistence_addon Tyre_addon 
          Electronic_addon thirt_party_liablity_premium manufacturer $ Party_type $ registration_date $ Car_number $
;    
data data_2018;
    infile file_4
        dlm=','
        firstobs = 2
        truncover
        ;
    length type $ 50;
    length manufacturer $ 50;
    length party_type $ 50;
    length car_number $ 20;
    length registration_date $ 10;
    input Policy_num mobile_num TxnID Year Type$ Base_Premium No_Claim_bonus_discount road_side_assistence_addon Tyre_addon 
          Electronic_addon thirt_party_liablity_premium manufacturer $ Party_type $ registration_date $ Car_number $
;    
data data_2019;
    infile file_5
        dlm=','
        firstobs = 2
        truncover
        ;
    length type $ 50;
    length manufacturer $ 50;
    length party_type $ 50;
    length car_number $ 20;
    length registration_date $ 10;
    input Policy_num mobile_num TxnID Year Type$ Base_Premium No_Claim_bonus_discount road_side_assistence_addon Tyre_addon 
          Electronic_addon thirt_party_liablity_premium manufacturer $ Party_type $ registration_date $ Car_number $
;    

data RTO;
    infile file_6
        dlm=','
        firstobs = 2
        truncover
        ;
        length rto_name $ 20;
        length rto_id $ 1;
        input rto_name $ rto_id $;
        

data car_policy_data;
    set data_2015 data_2016 data_2017 data_2018 data_2019;
    registration_date = input(registration_date, ddmmyy10.);
    
proc sql;
    create table car_policy_data_final as
           select a.*,
                  b.rto_name,
                  b.rto_id
           from car_policy_data as a
                  left join RTO as b
                         on b.rto_id = substr(a.car_number,4,1)
   ;
QUIT;
proc contents data =   car_policy_data_final;
proc print data = car_policy_data_final (obs = 10);
run;

SAS Connection established. Subprocess id is 22131



DATA ABOVE SEEMS FINE

ONWARDS TO ANSWERING A FEW QUESTIONS

CALCULATE YEARLY PREMIUM

In [2]:
proc sql;
select year,
       (sum(Base_Premium-No_Claim_bonus_discount+road_side_assistence_addon+Tyre_addon+Electronic_addon+thirt_party_liablity_premium)/1000000)
         as annual_premium_in_M
from car_policy_data_final
group by 1 
order by 1;
quit;
run;

Year,annual_premium_in_M
2015,87.8934
2016,83.97465
2017,84.40897
2018,84.48651
2019,85.01171


CALCULATE YEARLY AVERAGE AGE AS OF 31 JULY 2020 

In [3]:
proc sql;
        select year,
               count(*) as cnt_records,
               sum(abs(intck('year', input(registration_date,8.),input('31-07-2020', ddmmyy10.)))) as total_age,
               (calculated total_age/calculated cnt_records) as average_age,
               avg(abs(intck('year', input(registration_date,8.),input('31-07-2020', ddmmyy10.)))) as average_age
        from car_policy_data_final
              group by 1
;
quit;
run;

Year,cnt_records,total_age,average_age,average_age
2015,3526,211560,60,60
2016,3397,203820,60,60
2017,3374,202440,60,60
2018,3369,202140,60,60
2019,3411,204660,60,60
